In [2]:
#Download and unzip files
!pip3 install scikit-learn
!sudo apt-get install unzip
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'
!pip3 install https://download.pytorch.org/whl/cu100/torch-1.0.1-cp36-cp36m-linux_x86_64.whl
!pip3 install torch torchvision
!wget -P /root/input/ -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

Reading package lists... Done
Building dependency tree       
Reading state information... Done
unzip is already the newest version (6.0-21ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 25 not upgraded.
     |████████████████████████████████| 614.8MB 28kB/s 
ERROR: torchvision 0.6.0+cu101 has requirement torch==1.5.0, but you'll have torch 1.0.1 which is incompatible.
  Found existing installation: torch 1.5.0+cu101
    Uninstalling torch-1.5.0+cu101:
      Successfully uninstalled torch-1.5.0+cu101
--2020-05-02 22:28:45--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.29.246
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.29.246|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1647046227 (1.5G) [application/x-gzip]
Saving to: ‘/root/input/GoogleNews-vectors-negative300.bin.gz’

GoogleNews-vectors- 100%[===================>]   1.53G  35.5MB/s    in 31s 

In [3]:
#Verfiy file download
#models.py
import torch
device =  torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torch.utils.data import Dataset
import itertools
import csv
import pandas as pd
import spacy
from torchtext.vocab import GloVe
import re
from torchtext import data
import pickle
import torch.optim as optim
import gensim
import torchtext.vocab as vocab
from tqdm import tqdm_notebook
#Verify CUDA acceleration should print cuda:0
print(device)

cuda:0


In [4]:
# Mount your google drive. 
# Use this to save your PyTorch model for submission
from google.colab import drive
drive.mount('/content/gdrive')
#Test drive access. 
#You should have a test.txt under a new folder cis530_hw6 in your Google drive
!ls

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
gdrive	sample_data


In [5]:
from torchtext.data import Field
import spacy
from gensim.models import KeyedVectors

# Train Word Embeddings and save
 
EMBEDDING_FILE = '/root/input/GoogleNews-vectors-negative300.bin.gz' # from above
model_em = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)
weights = torch.FloatTensor(model_em.vectors)
t_embedding = nn.Embedding.from_pretrained(weights)


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
# NLP = spacy.load('en')
MAX_CHARS = 20000
MAX_VOCAB_SIZE = 25000
comment = data.Field(
    tokenize='spacy'
)
label = data.LabelField()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
def load_data(train_file, val_file, test_file):

    train = data.TabularDataset(
        path=train_file, format='csv', skip_header=True,
        fields=[
            ('label', label),
            ('text', comment),
        ])
    val = data.TabularDataset(
        path=val_file, format='csv', skip_header=True,
        fields=[
            ('label', label),
            ('text', comment),
        ])
    test = data.TabularDataset(
        path=test_file, format='csv', skip_header=True,
        fields=[
            ('label', label),
            ('text', comment),
        ])
    
    comment.build_vocab(train, vectors=GloVe(name='6B', dim=100),
                        max_size=MAX_VOCAB_SIZE,
                        min_freq=5,
                        unk_init=torch.Tensor.normal_
                        )#
          
    label.build_vocab(train)
    print(label.vocab.stoi)

    BATCH_SIZE = 64

    train_iterator, val_iterator, test_iterator = data.BucketIterator.splits(
        (train, val, test),
        batch_size=BATCH_SIZE,
        device=device,
        sort=False)
    # pickle.dump(train, open("train_iter.p", "wb"))
    # pickle.dump(val, open("val_iter.p", "wb"))
    # pickle.dump(test, open("test_iter.p", "wb"))
    return train_iterator, val_iterator, test_iterator


class CNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim,
                 dropout, pad_idx):
        super().__init__()

        self.embedding =  nn.Embedding(vocab_size, embedding_dim)

        self.convs = nn.ModuleList([
            nn.Conv2d(in_channels=1,
                      out_channels=n_filters,
                      kernel_size=(fs, embedding_dim))
            for fs in filter_sizes
        ])

        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)

        self.dropout = nn.Dropout(dropout)

    def forward(self, text):

        text = text.permute(1, 0)
        embedded = self.embedding(text)
        embedded = embedded.unsqueeze(1)
        conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        cat = self.dropout(torch.cat(pooled, dim=1))
        return self.fc(cat)

In [0]:
############################################################
# Reference Code
############################################################
def categorical_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    max_preds = preds.argmax(dim = 1, keepdim = True) # get the index of the max probability
    correct = max_preds.squeeze(1).eq(y)
    return correct.sum() / torch.FloatTensor([y.shape[0]]), max_preds, correct


def train(model, iterator, optimizer, criterion):
    epoch_loss = 0
    epoch_acc = 0

    model.train()

    for batch in iterator:
        preds=[]
        trues=[]
        optimizer.zero_grad()
        predictions = model(batch.text)
        loss = criterion(predictions, batch.label)
        acc,max_preds,correct = categorical_accuracy(predictions, batch.label)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        # f1_score
        preds += predictions.squeeze(1).argmax(dim=1).tolist()
        trues += batch.label.tolist()

    return epoch_loss / len(iterator), epoch_acc / len(iterator), preds, trues


def evaluate(model, iterator, criterion):
    epoch_loss = 0
    epoch_acc = 0
    preds=[]
    trues=[]
    model.eval()

    with torch.no_grad():
        for batch in iterator:
            predictions = model(batch.text)
            loss = criterion(predictions, batch.label)
            acc, max_pred, correct = categorical_accuracy(predictions, batch.label)
            epoch_loss += loss.item()
            epoch_acc += acc.item()
            # f1_score
            preds += predictions.squeeze(1).argmax(dim=1).tolist()
            trues += batch.label.tolist()

    return epoch_loss / len(iterator), epoch_acc / len(iterator), preds, trues

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs




In [8]:
# import os
# os.listdir("./gdrive/My Drive/530/")

FileNotFoundError: ignored

In [0]:
  
  train_iterator, val_iterator, test_iterator = load_data('./gdrive/My Drive/530/train.csv', './gdrive/My Drive/530/val.csv', 
                                                          './gdrive/My Drive/530/test.csv')
  INPUT_DIM = len(comment.vocab)# comment.vocab
  EMBEDDING_DIM = 300
  N_FILTERS = 300
  FILTER_SIZES = [3,4,5]
  #3,4,5
  OUTPUT_DIM = len(label.vocab)
  DROPOUT = 0.5
  PAD_IDX = comment.vocab.stoi[comment.pad_token]
  N_EPOCHS = 5

  model = CNN(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT, PAD_IDX)
  model.embedding = t_embedding
  #UNK_IDX = comment.vocab.stoi[comment.unk_token]

  #model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
  #model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)
 
  optimizer = optim.Adadelta(model.parameters(), weight_decay=1e-5) # apply l2-reg
  criterion = nn.CrossEntropyLoss()
  model = model.to(device)
  criterion = criterion.to(device)
  best_valid_loss = float('inf')
  train_preds=[]
  train_trues=[]
  val_preds =[]
  val_trues =[]
  for epoch in range(N_EPOCHS):

      start_time = time.time()
      train_loss, train_acc, train_pred, train_true = train(model, train_iterator, optimizer, criterion)
      valid_loss, valid_acc, val_pred, val_true = evaluate(model, val_iterator, criterion)
      val_preds.extend(val_pred)
      val_trues.extend(val_true)
      train_preds.extend(train_pred)
      train_trues.extend(train_true)

      end_time = time.time()

      epoch_mins, epoch_secs = epoch_time(start_time, end_time)

      if valid_loss < best_valid_loss:
          best_valid_loss = valid_loss
          torch.save(model.state_dict(), 'baseline-model.pt')

      print(f'Epoch: {epoch + 1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
      print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc * 100:.2f}%')
      print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc * 100:.2f}%')


.vector_cache/glove.6B.zip: 862MB [06:27, 2.22MB/s]                           
100%|█████████▉| 399886/400000 [00:13<00:00, 27221.57it/s]

defaultdict(<function _default_unk_index at 0x7fbc1ca1abf8>, {'0': 0, '1': 1, '-1': 2})


100%|█████████▉| 399886/400000 [00:30<00:00, 27221.57it/s]

Epoch: 01 | Epoch Time: 1m 11s
	Train Loss: 0.337 | Train Acc: 88.05%
	 Val. Loss: 0.198 |  Val. Acc: 93.52%
Epoch: 02 | Epoch Time: 1m 11s
	Train Loss: 0.228 | Train Acc: 92.51%
	 Val. Loss: 0.174 |  Val. Acc: 94.46%
Epoch: 03 | Epoch Time: 1m 11s
	Train Loss: 0.199 | Train Acc: 93.59%
	 Val. Loss: 0.166 |  Val. Acc: 94.82%
Epoch: 04 | Epoch Time: 1m 11s
	Train Loss: 0.183 | Train Acc: 94.16%
	 Val. Loss: 0.161 |  Val. Acc: 95.14%
Epoch: 05 | Epoch Time: 1m 12s
	Train Loss: 0.171 | Train Acc: 94.55%
	 Val. Loss: 0.159 |  Val. Acc: 95.20%
Epoch: 06 | Epoch Time: 1m 12s
	Train Loss: 0.163 | Train Acc: 94.83%
	 Val. Loss: 0.157 |  Val. Acc: 95.35%
Epoch: 07 | Epoch Time: 1m 12s
	Train Loss: 0.156 | Train Acc: 95.04%
	 Val. Loss: 0.161 |  Val. Acc: 95.35%
Epoch: 08 | Epoch Time: 1m 11s
	Train Loss: 0.151 | Train Acc: 95.22%
	 Val. Loss: 0.155 |  Val. Acc: 95.58%
Epoch: 09 | Epoch Time: 1m 12s
	Train Loss: 0.147 | Train Acc: 95.34%
	 Val. Loss: 0.149 |  Val. Acc: 95.65%
Epoch: 10 | Epoch T

In [0]:

test_loss, test_acc, test_preds, test_trues = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc * 100:.2f}%')
  
# f1 score
from sklearn.metrics import f1_score,recall_score, precision_score
#print(f1_score(preds, trues, average='weighted'))
# training 
print(f1_score(train_trues, train_preds, average='macro'))
print(precision_score(train_trues,train_preds,average='macro'))
print(recall_score(train_trues, train_preds,average='macro'))

# validation
print(f1_score(val_trues, val_preds, average='macro'))
print(precision_score(val_trues,val_preds,average='macro'))
print(recall_score(val_trues, val_preds,average='macro'))

#test 
print(f1_score(test_trues, test_preds, average='macro'))
print(precision_score(test_trues,test_preds,average='macro'))
print(recall_score(test_trues, test_preds,average='macro'))


'''
0.8812620211468666
0.8854992768977231
0.8774410774410774
0.9104255886331983
0.9304217405792423
0.8945494308819626
0.9200114337537629
0.9372097195318804
0.9060100274986641

300 5,5
0.9357373530580609

300 3,4,5
0.94
'''

Test Loss: 0.147 | Test Acc: 95.81%
0.9254674670554447
0.9312078929066577
0.9201562002274734
0.9373028159422666
0.9475678797007289
0.9282059324979764
0.9454422659326581
0.951540857305799
0.9397809398751455


'\n0.8812620211468666\n0.8854992768977231\n0.8774410774410774\n0.9104255886331983\n0.9304217405792423\n0.8945494308819626\n0.9200114337537629\n0.9372097195318804\n0.9060100274986641\n\n300 5,5\n0.9357373530580609\n\n300 3,4,5\n0.94\n'

In [0]:
print(len(test_preds))
print(len(test_trues))
for i in range(100):
  if test_preds[i]!=test_trues[i]:
    print(i,test_preds[i],test_trues[i])

235992
235992
0 1 0
6 2 1
40 2 1
49 0 2
69 0 2
83 0 1
84 0 2
98 0 2
